In [3]:
# test_script.py

import pandas as pd
import json
import requests
import os
import uuid
from collections import defaultdict
import re
import io
from django.http import HttpRequest, HttpResponse, JsonResponse
from views import create_comparison_table

API_KEY = "gYV8t7d6c5naotp67meIJyJRi6DksKv0VfPSQzEa"  # Replace with your API key
root_url = "https://developer.nrel.gov/api/reopt/stable"

# Disable warnings
import urllib3
urllib3.disable_warnings()

def test_create_comparison_table(run_uuids):
    # Create a mock request object
    request = HttpRequest()
    request.method = 'GET'
    
    # Mock API key and root URL for the API requests
    request.META['API_KEY'] = API_KEY
    request.META['root_url'] = root_url

    # Mock user UUID
    user_uuid = str(uuid.uuid4())

    # Replace the actual fetch_raw_data method to use the provided run UUIDs
    def fetch_raw_data(request, run_uuid):
        results_url = f"{root_url}/job/{run_uuid}/results/?api_key={API_KEY}"
        response = requests.get(results_url, verify=False)
        response.raise_for_status()
        result_data = response.json()
        processed_data = sum_vectors(result_data)  # Summing vectors into a single value
        return processed_data

    # Replace the actual get_raw_data method to use the provided run UUIDs
    def get_raw_data(api_metas, request):
        full_summary_dict = {"scenarios": []}
        for m in api_metas:
            scenario_data = {
                "run_uuid": str(m['run_uuid']),
                "status": m['status'],
                "created": str(m['created']),
                "full_data": fetch_raw_data(request, m['run_uuid'])
            }
            full_summary_dict["scenarios"].append(scenario_data)
        return full_summary_dict

    # Mock API meta data
    api_metas = [{"run_uuid": run_uuid, "status": "completed", "created": "2024-08-08"} for run_uuid in run_uuids]

    # Call the create_comparison_table function with the mock request and user UUID
    response = create_comparison_table(request, user_uuid)

    # Check the response type and print the appropriate result
    if isinstance(response, HttpResponse):
        if response.status_code == 200:
            # Save the response content to an Excel file
            with open("comparison_table.xlsx", "wb") as f:
                f.write(response.content)
            print("Comparison table saved to 'comparison_table.xlsx'.")
        else:
            print(f"Error: {response.status_code} - {response.content.decode()}")
    elif isinstance(response, JsonResponse):
        print(json.dumps(response.json(), indent=4))
    else:
        print("Unexpected response type.")

# Define the runrun_uuid_1 = "4043a50f-52b9-482a-90dd-8f7ea417182a"
run_uuid_2 = "3ccb973a-e9ed-405e-bb41-4fcb0f4bb9a5"
run_uuids = [run_uuid_1, run_uuid_2] UUIDs


# Run the test
test_create_comparison_table(run_uuids)


ModuleNotFoundError: No module named 'reo'